# 패키지 로딩

In [1]:
import numpy as np
import pandas as pd

import warnings

warnings.filterwarnings("ignore")

# 데이터 로딩

## 서울시 구별 남녀 인구수
- 출처: 통계청 행정구역(시군구)별 성별 인구수
- 데이터 분류: 월별 데이터
- 기간: 2014년 3월 ~ 2024년 2월 (120개월, 10년치)


In [102]:
df1 = pd.read_csv("./data/서울시 구단위 남녀 인구수.csv", header=[0, 1], encoding="cp949")
df1

시점       종로구                  중구                 용산구            \
          시점 남자인구수 (명) 여자인구수 (명) 남자인구수 (명) 여자인구수 (명) 남자인구수 (명) 여자인구수 (명)   
0    2014.03     79153     80398     65029     64911    116309    123072   
1    2014.04     78943     80366     64922     64818    116127    122874   
2    2014.05     78821     80269     64842     64801    116034    122886   
3    2014.06     78678     80143     64745     64685    115857    122666   
4    2014.07     78394     79990     64638     64598    115651    122400   
..       ...       ...       ...       ...       ...       ...       ...   
115  2023.10     67466     72140     58720     62580    103398    111999   
116  2023.11     67393     72104     58631     62599    102925    111456   
117  2023.12     67306     72111     58659     62653    102312    110839   
118  2024.01     67240     72138     58651     62671    101793    110382   
119  2024.02     67266     72205     58711     62708    101262    109985   

          성동구                 광진구  ...       관악구                 서초구  \
    남자인구수 (명) 여자인구수 (명) 남자인구수 (명)  ... 남자인구수 (명) 여자인구수 (명) 남자인구수 (명)   
0      149232    150184    181376  ...    262185    255598    214053   
1      149265    150337    181154  ...    262211    255479    213806   
2      149203    150318    180923  ...    261951    255363    213802   
3      149082    150246    180666  ...    261623    255125    214062   
4      148906    150133    180365  ...    261331    254772    214625   
..        ...       ...       ...  ...       ...       ...       ...   
115    134979    143056    161662  ...    243673    240286    194384   
116    134760    142941    161430  ...    242985    239707    194526   
117    134519    142842    161277  ...    242651    239305    194291   
118    134390    142700    161154  ...    242598    239274    194231   
119    134278    142674    161022  ...    242703    239131    194232   

                    강남구                 송파구                 강동구            
    여자인구수 (명) 남자인구수 (명) 여자인구수 (명) 남자인구수 (명) 여자인구수 (명) 남자인구수 (명) 여자인구수 (명)  
0      229619    271343    293959    328684    343612    240832    241375  
1      229424    272032    294722    328202    343229    240526    241073  
2      229551    272775    295503    327978    342999    240192    240886  
3      229878    272991    295876    327710    342833    240008    240797  
4      230563    273642    296689    327287    342586    239638    240425  
..        ...       ...       ...       ...       ...       ...       ...  
115    213404    258897    282668    314985    340315    223930    234362  
116    213572    258700    282430    314665    340180    224240    234584  
117    213373    260520    284353    314347    339819    224423    234744  
118    213537    262618    286536    314189    339800    224450    234939  
119    213764    264695    288860    314043    339879    224986    235529  

[120 rows x 51 columns]

## 서울시 연령대별 남녀 인구수
- 출처: 통계청 시군구 성 연령 5세 별 주민등록연앙인구
- 데이터 분류: 연간 데이터
- 기간: 2014년 ~ 2023년 (10년치)

### 남자 인구수

In [62]:
df2 = pd.read_csv("./data/서울시 구단위 연령별 인구수_남.csv", index_col=[0, 1],encoding="cp949")
df2

종로구      중구     용산구      성동구      광진구     동대문구      중랑구  \
시점   연령별                                                                    
2014 0 - 4세    2323.5  2396.5  4647.0   6336.0   7004.5   7055.5   8188.0   
     5 - 9세    2635.0  2002.5  4317.0   5819.0   6641.0   6728.5   7464.0   
     10 - 14세  3587.0  2211.5  4874.5   6159.5   8402.0   7613.5   8906.0   
     15 - 19세  4756.5  3073.0  6236.0   7955.5  11172.0   9778.0  12599.0   
     20 - 24세  5683.5  4253.0  7034.5  10507.5  13424.0  13270.5  14814.5   
...               ...     ...     ...      ...      ...      ...      ...   
2023 80 - 84세  1789.0  1445.0  2226.0   2741.5   3092.0   3882.0   4169.0   
     85 - 89세   824.0   568.0  1070.0   1103.0   1197.5   1571.0   1586.0   
     90 - 94세   233.0   177.5   331.0    271.0    303.5    371.0    340.0   
     95 - 99세    53.5    40.0    58.5     49.5     60.5     54.0     69.5   
     100세 이상      8.5     5.5     7.0     11.0     12.0     10.0      7.0   

                   성북구      강북구      도봉구  ...      강서구      구로구     금천구  \
시점   연령별                                  ...                             
2014 0 - 4세     9813.0   6353.5   6921.0  ...  13225.5  10672.5  4503.5   
     5 - 9세    10355.0   6007.0   6953.5  ...  11911.0   9201.5  4099.5   
     10 - 14세  11366.0   7479.0   8721.5  ...  13605.5   9310.5  4928.5   
     15 - 19세  13328.0   9843.5  11778.0  ...  17184.5  11547.5  7077.5   
     20 - 24세  16137.0  10972.5  12907.5  ...  19673.5  13505.5  8449.0   
...                ...      ...      ...  ...      ...      ...     ...   
2023 80 - 84세   4757.0   4090.0   4071.0  ...   4760.5   4198.5  2399.5   
     85 - 89세   1918.0   1646.0   1564.0  ...   1852.5   1552.5   853.0   
     90 - 94세    486.5    359.5    406.5  ...    510.5    363.0   220.0   
     95 - 99세     68.0     59.0     83.0  ...     88.0     52.0    30.0   
     100세 이상      10.5     11.5     10.5  ...     14.0      6.5     8.0   

                  영등포구      동작구      관악구      서초구      강남구      송파구      강동구  
시점   연령별                                                                      
2014 0 - 4세     8248.0   8788.0   9393.0  10577.5  10977.5  14877.0  10386.0  
     5 - 9세     7452.5   7982.5   8477.0  10777.5  11059.0  14830.5  10068.0  
     10 - 14세   7882.5   8544.0   9425.5  12443.5  15536.0  17908.5  11994.0  
     15 - 19세  10040.0  10963.0  12392.5  14538.5  22022.5  21583.0  16067.5  
     20 - 24세  12050.0  14382.5  18458.5  14248.0  19957.0  23042.0  17462.5  
...                ...      ...      ...      ...      ...      ...      ...  
2023 80 - 84세   3736.5   4138.5   5166.0   3577.5   4548.0   5029.5   3822.5  
     85 - 89세   1651.5   1638.0   1824.0   1501.0   1772.0   1982.5   1365.0  
     90 - 94세    420.5    407.0    389.0    416.0    491.0    522.0    350.5  
     95 - 99세     66.0     79.0     52.0     82.5     99.5    103.0     61.0  
     100세 이상      10.0     11.5      9.5      8.5     13.5     14.5     13.0  

[210 rows x 25 columns]

### 여자 인구수

In [63]:
df2 = pd.read_csv("./data/서울시 구단위 연령별 인구수_여.csv", index_col=[0, 1],encoding="cp949")
df2

종로구      중구     용산구     성동구      광진구     동대문구      중랑구  \
시점   연령별                                                                   
2014 0 - 4세    2175.5  2276.5  4521.5  6052.0   6633.0   6691.0   7751.0   
     5 - 9세    2402.5  2004.5  4273.0  5532.0   6445.0   6391.0   7108.0   
     10 - 14세  3296.5  2079.0  4629.5  6107.0   7736.0   7208.5   8367.5   
     15 - 19세  4362.5  2872.0  5579.0  7693.5  10620.0   9264.0  11467.0   
     20 - 24세  5593.5  4138.5  7295.5  9937.0  13674.0  12571.0  14325.0   
...               ...     ...     ...     ...      ...      ...      ...   
2023 80 - 84세  2504.0  2134.0  3351.5  3991.5   4104.5   5422.0   5809.5   
     85 - 89세  1359.0  1232.0  2028.5  2143.0   2178.5   2750.0   2812.0   
     90 - 94세   491.0   498.5   757.0   720.0    780.0    918.5   1060.5   
     95 - 99세   136.0   121.5   176.0   155.5    196.5    211.5    270.5   
     100세 이상     22.5    20.0    37.5    29.5     27.5     45.0     36.0   

                   성북구      강북구      도봉구  ...      강서구      구로구     금천구  \
시점   연령별                                  ...                             
2014 0 - 4세     9382.0   6022.0   6299.5  ...  12832.5  10309.5  4266.5   
     5 - 9세     9703.5   5707.0   6528.5  ...  11411.0   8608.5  3998.5   
     10 - 14세  10618.0   6815.5   8243.0  ...  12874.0   8630.5  4691.0   
     15 - 19세  12618.0   9042.0  10708.0  ...  15917.0  10565.0  6420.0   
     20 - 24세  16503.5  10742.0  12068.5  ...  19174.5  12928.5  7939.5   
...                ...      ...      ...  ...      ...      ...     ...   
2023 80 - 84세   6812.5   5921.0   5602.0  ...   6888.0   5449.0  3184.5   
     85 - 89세   3433.5   2669.0   2862.0  ...   3842.0   2734.5  1547.0   
     90 - 94세   1240.5    884.5   1095.5  ...   1592.5    973.0   542.0   
     95 - 99세    324.5    217.5    254.5  ...    423.5    249.0   124.5   
     100세 이상      38.5     29.0     37.0  ...     59.0     34.5    22.0   

                  영등포구      동작구      관악구      서초구      강남구      송파구      강동구  
시점   연령별                                                                      
2014 0 - 4세     7975.5   8231.0   9036.0   9991.5  10310.0  14120.0   9755.5  
     5 - 9세     6916.5   7410.0   8042.5  10256.5  10426.0  13882.5   9227.5  
     10 - 14세   7118.5   7961.5   8665.5  11927.0  13909.0  16690.0  10747.5  
     15 - 19세   9227.5  10192.5  11576.5  13480.5  19238.0  20477.0  13908.5  
     20 - 24세  12128.0  14280.5  19390.5  14052.0  19917.0  22116.5  15696.0  
...                ...      ...      ...      ...      ...      ...      ...  
2023 80 - 84세   5026.0   5231.5   6462.5   3978.0   5016.0   6147.0   4861.0  
     85 - 89세   2711.0   2820.5   2929.5   2214.5   2798.5   3505.5   2639.5  
     90 - 94세    961.0   1042.5   1042.0    943.5   1286.0   1606.5   1177.0  
     95 - 99세    213.5    257.0    250.0    271.0    365.0    401.5    291.0  
     100세 이상      33.5     34.5     41.5     42.0     58.5     65.0     56.0  

[210 rows x 25 columns]

# 데이터 전처리

## 서울시 구단위 남녀 인구수
- 시점 pd.datetime으로 변경

In [86]:
# 데이터 타입 확인: float64
df1[("시점", "시점")].info()

<class 'pandas.core.series.Series'>
RangeIndex: 120 entries, 0 to 119
Series name: ('시점', '시점')
Non-Null Count  Dtype  
--------------  -----  
120 non-null    float64
dtypes: float64(1)
memory usage: 1.1 KB


In [117]:
# float -> stirng -> pd.to_datetime()
df1[("시점", "시점")] = df1[("시점", "시점")].apply(lambda x: pd.to_datetime(str(x)))

df1 = df1.set_index(df1[("시점", "시점")]) 
df1 = df1.drop(('시점', '시점'), axis = 1)
df1.index.name = None
df1

종로구                  중구                 용산구            \
           남자인구수 (명) 여자인구수 (명) 남자인구수 (명) 여자인구수 (명) 남자인구수 (명) 여자인구수 (명)   
2014-03-01     79153     80398     65029     64911    116309    123072   
2014-04-01     78943     80366     64922     64818    116127    122874   
2014-05-01     78821     80269     64842     64801    116034    122886   
2014-06-01     78678     80143     64745     64685    115857    122666   
2014-07-01     78394     79990     64638     64598    115651    122400   
...              ...       ...       ...       ...       ...       ...   
2023-01-01     67466     72140     58720     62580    103398    111999   
2023-11-01     67393     72104     58631     62599    102925    111456   
2023-12-01     67306     72111     58659     62653    102312    110839   
2024-01-01     67240     72138     58651     62671    101793    110382   
2024-02-01     67266     72205     58711     62708    101262    109985   

                 성동구                 광진구            ...       관악구            \
           남자인구수 (명) 여자인구수 (명) 남자인구수 (명) 여자인구수 (명)  ... 남자인구수 (명) 여자인구수 (명)   
2014-03-01    149232    150184    181376    186949  ...    262185    255598   
2014-04-01    149265    150337    181154    186739  ...    262211    255479   
2014-05-01    149203    150318    180923    186468  ...    261951    255363   
2014-06-01    149082    150246    180666    186273  ...    261623    255125   
2014-07-01    148906    150133    180365    185943  ...    261331    254772   
...              ...       ...       ...       ...  ...       ...       ...   
2023-01-01    134979    143056    161662    174632  ...    243673    240286   
2023-11-01    134760    142941    161430    174424  ...    242985    239707   
2023-12-01    134519    142842    161277    174277  ...    242651    239305   
2024-01-01    134390    142700    161154    174181  ...    242598    239274   
2024-02-01    134278    142674    161022    174178  ...    242703    239131   

                 서초구                 강남구                 송파구            \
           남자인구수 (명) 여자인구수 (명) 남자인구수 (명) 여자인구수 (명) 남자인구수 (명) 여자인구수 (명)   
2014-03-01    214053    229619    271343    293959    328684    343612   
2014-04-01    213806    229424    272032    294722    328202    343229   
2014-05-01    213802    229551    272775    295503    327978    342999   
2014-06-01    214062    229878    272991    295876    327710    342833   
2014-07-01    214625    230563    273642    296689    327287    342586   
...              ...       ...       ...       ...       ...       ...   
2023-01-01    194384    213404    258897    282668    314985    340315   
2023-11-01    194526    213572    258700    282430    314665    340180   
2023-12-01    194291    213373    260520    284353    314347    339819   
2024-01-01    194231    213537    262618    286536    314189    339800   
2024-02-01    194232    213764    264695    288860    314043    339879   

                 강동구            
           남자인구수 (명) 여자인구수 (명)  
2014-03-01    240832    241375  
2014-04-01    240526    241073  
2014-05-01    240192    240886  
2014-06-01    240008    240797  
2014-07-01    239638    240425  
...              ...       ...  
2023-01-01    223930    234362  
2023-11-01    224240    234584  
2023-12-01    224423    234744  
2024-01-01    224450    234939  
2024-02-01    224986    235529  

[120 rows x 50 columns]